# Speed 

In [1]:
import numpy as np
import cupy as cp
import numba as nb

In [2]:
size = (4096, 4096)

In [3]:
input = np.random.normal(size = size).astype(np.float32)

In [4]:
%timeit -n 1 -r 1 output = np.sort(input)

163 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [5]:
from cupyx.profiler import benchmark

input_gpu = cp.array(input)
execution_gpu = benchmark(cp.sort, (input_gpu,), n_repeat = 10)
gpu_avg_time = np.mean(execution_gpu.gpu_times)
print(f"{gpu_avg_time:.6f} s")

0.012920 s


In [9]:
speedup = 0.16 / gpu_avg_time
speedup

np.float64(12.383713741649416)

# Using Numba

In [101]:
import numpy as np
import matplotlib.pyplot as plt
import math
import numba as nb
from numba import jit, cuda
import cupy as cp

## CPU

In [67]:
def find_all_primes_cpu(upper):
    primes = []
    for i in range(2, upper+1):
        if i == 2:
            primes.append(i)
        elif i % 2 == 1:
            for k in range(2, int(np.ceil(np.sqrt(i)+1))):
                if i % k == 0:
                    break
            else: 
            # this else portion only runs when it 
            # doesn't break the inner loop
            # for-else loops
                primes.append(i)
    return primes

In [42]:
for j in range(2, 273):
    print(f"upper bound = {j}: {find_all_primes_cpu(j)}")

upper bound = 2: [2]
upper bound = 3: [2, 3]
upper bound = 4: [2, 3]
upper bound = 5: [2, 3, 5]
upper bound = 6: [2, 3, 5]
upper bound = 7: [2, 3, 5, 7]
upper bound = 8: [2, 3, 5, 7]
upper bound = 9: [2, 3, 5, 7]
upper bound = 10: [2, 3, 5, 7]
upper bound = 11: [2, 3, 5, 7, 11]
upper bound = 12: [2, 3, 5, 7, 11]
upper bound = 13: [2, 3, 5, 7, 11, 13]
upper bound = 14: [2, 3, 5, 7, 11, 13]
upper bound = 15: [2, 3, 5, 7, 11, 13]
upper bound = 16: [2, 3, 5, 7, 11, 13]
upper bound = 17: [2, 3, 5, 7, 11, 13, 17]
upper bound = 18: [2, 3, 5, 7, 11, 13, 17]
upper bound = 19: [2, 3, 5, 7, 11, 13, 17, 19]
upper bound = 20: [2, 3, 5, 7, 11, 13, 17, 19]
upper bound = 21: [2, 3, 5, 7, 11, 13, 17, 19]
upper bound = 22: [2, 3, 5, 7, 11, 13, 17, 19]
upper bound = 23: [2, 3, 5, 7, 11, 13, 17, 19, 23]
upper bound = 24: [2, 3, 5, 7, 11, 13, 17, 19, 23]
upper bound = 25: [2, 3, 5, 7, 11, 13, 17, 19, 23]
upper bound = 26: [2, 3, 5, 7, 11, 13, 17, 19, 23]
upper bound = 27: [2, 3, 5, 7, 11, 13, 17, 19, 23]
u

In [43]:
oeis = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271]

In [44]:
find_all_primes_cpu(272) == oeis

True

In [50]:
%timeit -n 10 -r 1 find_all_primes_cpu(1_000_000)

1.36 s ± 0 ns per loop (mean ± std. dev. of 1 run, 10 loops each)


In [100]:
# @jit(nopython = True) converts to machine code

@jit(nopython = True)
def jit_find_all_primes_cpu(upper):
    primes = []
    for i in range(2, upper+1):
        if i == 2:
            primes.append(i)
        elif i % 2 == 1:
            for k in range(2, math.ceil(math.sqrt(i)+1)):
                if i % k == 0:
                    break
            else: 
            # this else portion only runs when it 
            # doesn't break the inner loop
            # for-else loops
                primes.append(i)
    return primes

In [54]:
%timeit -n 10 -r 1 jit_find_all_primes_cpu(1_000_000)

158 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 10 loops each)


## GPU

In [88]:
@cuda.jit
def check_prime_gpu_kernel(num, result):
    # result is an array, kernels cannot return anything
    # so give an np or cp array to modify
    result[0] = num

    for i in range(2, int(math.ceil(math.sqrt(num)+1)) ):
        if num % i == 0:
            result[0] = 0
            break

# can only use math package, cannot use numpy or cupy 

In [92]:
# uses one block and one thread form the block [block, thread]
arr = np.zeros(1, cp.int64)
# it changes the array u give
check_prime_gpu_kernel[1, 1](9, arr)

In [93]:
arr[0]

np.int64(0)

In [95]:
def find_all_primes_cpu_and_gpu(upper):
    primes = []
    for i in range(2, upper+1):
        if i == 2:
            primes.append(i)
        elif i % 2 == 1:
            prime_array = np.zeros(1, np.int64)
            check_prime_gpu_kernel[1,1](i, prime_array)
            
            if prime_array[0] != 0:
                primes.append(i)
    return primes

In [98]:
%timeit -n 10 -r 1 find_all_primes_cpu_and_gpu(10_000)

1.25 s ± 0 ns per loop (mean ± std. dev. of 1 run, 10 loops each)


In [97]:
for j in range(2, 273):
    print(f"upper bound = {j}: {find_all_primes_cpu_and_gpu(j)}")

upper bound = 2: [2]
upper bound = 3: [2, 3]
upper bound = 4: [2, 3]
upper bound = 5: [2, 3, 5]
upper bound = 6: [2, 3, 5]
upper bound = 7: [2, 3, 5, 7]
upper bound = 8: [2, 3, 5, 7]
upper bound = 9: [2, 3, 5, 7]
upper bound = 10: [2, 3, 5, 7]
upper bound = 11: [2, 3, 5, 7, 11]
upper bound = 12: [2, 3, 5, 7, 11]
upper bound = 13: [2, 3, 5, 7, 11, 13]
upper bound = 14: [2, 3, 5, 7, 11, 13]
upper bound = 15: [2, 3, 5, 7, 11, 13]
upper bound = 16: [2, 3, 5, 7, 11, 13]
upper bound = 17: [2, 3, 5, 7, 11, 13, 17]
upper bound = 18: [2, 3, 5, 7, 11, 13, 17]
upper bound = 19: [2, 3, 5, 7, 11, 13, 17, 19]
upper bound = 20: [2, 3, 5, 7, 11, 13, 17, 19]
upper bound = 21: [2, 3, 5, 7, 11, 13, 17, 19]
upper bound = 22: [2, 3, 5, 7, 11, 13, 17, 19]
upper bound = 23: [2, 3, 5, 7, 11, 13, 17, 19, 23]
upper bound = 24: [2, 3, 5, 7, 11, 13, 17, 19, 23]
upper bound = 25: [2, 3, 5, 7, 11, 13, 17, 19, 23]
upper bound = 26: [2, 3, 5, 7, 11, 13, 17, 19, 23]
upper bound = 27: [2, 3, 5, 7, 11, 13, 17, 19, 23]
u

In [99]:
find_all_primes_cpu_and_gpu(272) == oeis

/home/alvinng44/.local/share/mamba/envs/alyssa/lib/python3.12/site-packages/numba_cuda/numba/cuda/cudadrv/devicearray.py:937: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


True

In [124]:
# @nb.vectorize(["output_type(input_type)"], target = "cuda")
# @nb.vectorize(["int32(int32)"], target = "cuda")

@nb.vectorize(["int32(int32)"], target = "cuda")
def check_prime_gpu(num):
    if num == 1:
        return 0
    for i in range(2, math.ceil(math.sqrt(num)) + 1):
        if num != i:
            if num % i == 0:
                return 0
    return num

# returns 0 for that number in the array if composite

In [114]:
 for i in range(2, math.ceil(math.sqrt(2)) + 1):
     print(i)

2


In [118]:
%timeit -n 10 -r 1 check_prime_gpu(np.arange(0, 1_000_000, dtype = np.int32))

25.5 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 10 loops each)


In [125]:
primes_array = check_prime_gpu(np.arange(0, 1_000_000, dtype = np.int32))

In [126]:
primes_array

array([0, 0, 2, ..., 0, 0, 0], shape=(1000000,), dtype=int32)

In [128]:
primes_array != 0

array([False, False,  True, ..., False, False, False], shape=(1000000,))

In [129]:
primes_array[primes_array != 0]

array([     2,      3,      5, ..., 999961, 999979, 999983],
      shape=(78498,), dtype=int32)